## Model Building

In [1]:
import pandas as pd
import pickle

In [2]:
import demand_preprocessing_script as dps
import crowd_preprocessing_script as cps
import input_train 
import run_model
import append_holiday

In [3]:
address_file  = 'iso_address.csv'
data_filename = 'Mosaic Bay.json.pickle'
fc_id = '62ad73fdb3ece857aa492b60'
rest_id = '62ad77d1da7471469bbb06f9'
menu_id = 'Chapati'
type1 = 'demand'
type2 = 'crowd'
frq = "20min"

start_date = '2022-12-20 00:00:00+05:30'
end_date = '2022-12-27 23:00:00+05:30'
#data_filename = 'Mosaic Bay.json.pickle'

In [4]:
#df = dps.preprocessing(data_filename,'62aec058d7b2730639cc244c','Aloo Paratha with Curd')
df = dps.preprocessing(data_filename,rest_id,menu_id,frq)

In [ ]:
df.head(5)

,placedtime,menuitemname,holiday,crowd_count,count
89,2022-11-01 00:00:00+05:30,Chapati,1,0,0.0
319,2022-11-01 00:20:00+05:30,Chapati,1,0,0.0
470,2022-11-01 00:40:00+05:30,Chapati,1,0,0.0
863,2022-11-01 01:00:00+05:30,Chapati,1,0,0.0
1051,2022-11-01 01:20:00+05:30,Chapati,1,0,0.0


In [ ]:
df.resample("D", on = 'placedtime').sum().head(5)

,holiday,crowd_count,count
placedtime,,,
2022-11-01 00:00:00+05:30,72,0,0.0
2022-11-02 00:00:00+05:30,0,503,84.0
2022-11-03 00:00:00+05:30,0,578,69.0
2022-11-04 00:00:00+05:30,0,598,102.0
2022-11-05 00:00:00+05:30,0,404,94.0


In [ ]:
df_final = input_train.encode_extract(df)

In [ ]:
df_final = df_final[['holiday',
                    'min_sin',
                    'min_cos',
                    'hour_sin',
                    'hour_cos',
                    'date_sin',
                    'date_cos',
                    'month_sin',
                    'month_cos',
                    'dayofweek_sin',
                    'dayofweek_cos',
                    'crowd_count',
                    'count']]

In [ ]:
run_model.output_model(df_final,fc_id,rest_id,menu_id,type1)

0.671947451013113


In [37]:
# Getting crowd count
crowd_df = cps.crowd_num(data_filename,frq)
crowd_df = crowd_df.reset_index()

In [38]:
crowd_df  = append_holiday.add_holiday(crowd_df, fc_id)

In [39]:
crowd_df_final = input_train.encode_extract(crowd_df)

In [40]:
crowd_df_final = crowd_df_final[['holiday',
                    'min_sin',
                    'min_cos',
                    'hour_sin',
                    'hour_cos',
                    'date_sin',
                    'date_cos',
                    'month_sin',
                    'month_cos',
                    'dayofweek_sin',
                    'dayofweek_cos',
                    'restaurantcustomer'
                    ]]

In [41]:
run_model.output_model(crowd_df_final,fc_id,rest_id,menu_id,type2)

0.8044824643181393


## Forecasting

In [7]:
import pandas as pd
import append_holiday

In [8]:
f_new = pd.date_range(start_date, end_date, freq=frq).to_frame(index = False)
f_new = f_new.rename(columns = {0:'placedtime'})

In [9]:
f_new = append_holiday.add_holiday(f_new, fc_id)

In [10]:
f_new_final = input_train.encode_extract(f_new)

In [11]:
import pickle

In [12]:
pickle_file_crowd = "C:\GK\Project\Demand_estimation\data\\ver2\model_pickles\crowd\\"+fc_id+'_'+rest_id+'_'+menu_id+'.pickle'

In [13]:
prediction_crowd = pickle.load(open(pickle_file_crowd, 'rb')).predict(f_new_final)

In [14]:
for i in range(len(prediction_crowd)):
        if prediction_crowd[i]<0:
            prediction_crowd[i]=0
        else:
            prediction_crowd[i]=round(prediction_crowd[i])
    
crowd_pred_df = pd.DataFrame(prediction_crowd)
crowd_pred_df.index = f_new_final.index
crowd_pred_df = crowd_pred_df.rename(columns = {0:'crowd_count'})
crowd_pred_df = crowd_pred_df.reset_index()

In [15]:
crowd_pred_df = append_holiday.add_holiday(crowd_pred_df,fc_id)

In [16]:
demand_pred_final = input_train.encode_extract(crowd_pred_df)

In [17]:
demand_pred_final = demand_pred_final[['holiday',
                    'min_sin',
                    'min_cos',
                    'hour_sin',
                    'hour_cos',
                    'date_sin',
                    'date_cos',
                    'month_sin',
                    'month_cos',
                    'dayofweek_sin',
                    'dayofweek_cos',
                    'crowd_count']]

In [18]:
pickle_file_demand = "C:\GK\Project\Demand_estimation\data\\ver2\model_pickles\demand\\"+fc_id+'_'+rest_id+'_'+menu_id+'.pickle'

In [19]:
prediction_demand = pickle.load(open(pickle_file_demand, 'rb')).predict(demand_pred_final)

In [20]:
for i in range(len(prediction_demand)):
        if prediction_demand[i]<0:
            prediction_demand[i]=0
        else:
            prediction_demand[i]=round(prediction_demand[i])
    
demand_pred_df = pd.DataFrame(prediction_demand)
demand_pred_df.index = f_new_final.index
demand_pred_df = demand_pred_df.rename(columns = {0:'demand_count'})
demand_pred_df = demand_pred_df.reset_index()

In [21]:
demand_pred_df

,placedtime,demand_count
0,2022-12-20 00:00:00+05:30,0.0
1,2022-12-20 00:20:00+05:30,0.0
2,2022-12-20 00:40:00+05:30,0.0
3,2022-12-20 01:00:00+05:30,0.0
4,2022-12-20 01:20:00+05:30,0.0
...,...,...
569,2022-12-27 21:40:00+05:30,0.0
570,2022-12-27 22:00:00+05:30,0.0
571,2022-12-27 22:20:00+05:30,0.0
572,2022-12-27 22:40:00+05:30,0.0


In [23]:
demand_pred_df = demand_pred_df.resample('D',on = 'placedtime').sum()
demand_pred_df.index = demand_pred_df.index.date
demand_pred_df

,demand_count
2022-12-20,218.0
2022-12-21,219.0
2022-12-22,234.0
2022-12-23,211.0
2022-12-24,141.0
2022-12-25,41.0
2022-12-26,175.0
2022-12-27,185.0


In [29]:
demand_pred_df.to_csv('output.csv')

In [1]:
import api_input

In [2]:
data = {
  "start_date": "2022-12-20 00:00:00+05:30",
  "end_date": "2022-12-27 23:00:00+05:30",
  "fc_id": "62ad73fdb3ece857aa492b60",
  "rest_id": "62ad807dc4afda4b710aac1c",
  "menu_id": "Ginger tea"
}

In [3]:
output = api_input.date_input(data['start_date'], data['end_date'],data['fc_id'],data['rest_id'],data['menu_id'])

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\GK\\Project\\Demand_estimation\\data\\ver2\\model_pickle\\crowd\\62ad73fdb3ece857aa492b60_62ad807dc4afda4b710aac1c_Ginger tea.pickle'

In [ ]:
output.placedtime = output.placedtime.map(str)
output_dict = output.to_dict()
output_dict


{'placedtime': {'2022-12-20 00:00:00+05:30': '2022-12-20 00:00:00+05:30',
  '2022-12-20 00:20:00+05:30': '2022-12-20 00:20:00+05:30',
  '2022-12-20 00:40:00+05:30': '2022-12-20 00:40:00+05:30',
  '2022-12-20 01:00:00+05:30': '2022-12-20 01:00:00+05:30',
  '2022-12-20 01:20:00+05:30': '2022-12-20 01:20:00+05:30',
  '2022-12-20 01:40:00+05:30': '2022-12-20 01:40:00+05:30',
  '2022-12-20 02:00:00+05:30': '2022-12-20 02:00:00+05:30',
  '2022-12-20 02:20:00+05:30': '2022-12-20 02:20:00+05:30',
  '2022-12-20 02:40:00+05:30': '2022-12-20 02:40:00+05:30',
  '2022-12-20 03:00:00+05:30': '2022-12-20 03:00:00+05:30',
  '2022-12-20 03:20:00+05:30': '2022-12-20 03:20:00+05:30',
  '2022-12-20 03:40:00+05:30': '2022-12-20 03:40:00+05:30',
  '2022-12-20 04:00:00+05:30': '2022-12-20 04:00:00+05:30',
  '2022-12-20 04:20:00+05:30': '2022-12-20 04:20:00+05:30',
  '2022-12-20 04:40:00+05:30': '2022-12-20 04:40:00+05:30',
  '2022-12-20 05:00:00+05:30': '2022-12-20 05:00:00+05:30',
  '2022-12-20 05:20:00+05: